In [74]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from renetti.ws.spiders.types import ScrapedEquipment

In [6]:
url = "https://atlantisstrength.com/gym-equipment/"
base_url = "https://atlantisstrength.com"

In [12]:
async with async_playwright() as playwright:
    async with await playwright.chromium.launch(headless=False) as browser:
        async with await browser.new_context() as context:
            async with await context.new_page() as page:
                urls = []
                await page.goto(url=url)
                while True:
                    await page.wait_for_timeout(3000)
                    html = await page.content()
                    soup = BeautifulSoup(markup=html, features="html.parser")
                    a_tags = soup.findAll("a", class_="c-equipCards")
                    urls += [a.get('href') for a in a_tags]

                    try:
                        await page.wait_for_timeout(2000)

                        button = await page.wait_for_selector('button.c-Pagination__content--next', timeout=2000, state='visible')

                        # Ensure the button is in view before clicking
                        await button.scroll_into_view_if_needed()

                        # Click the button and wait for navigation or content change
                        await button.click()

                        # Wait a bit for content to load after the click
                        await page.wait_for_timeout(2000)
                    except Exception as exp:
                        print(exp)
                        break

Page.wait_for_timeout: Target page, context or browser has been closed


In [25]:
url = "https://atlantisstrength.com/gym-equipment/b157"

In [72]:
async with async_playwright() as playwright:
    async with await playwright.chromium.launch(headless=False) as browser:
        async with await browser.new_context() as context:
            async with await context.new_page() as page:
                await page.goto(url=url)
                await page.wait_for_timeout(2000)
                html = await page.content()
                soup = BeautifulSoup(markup=html, features="html.parser")

                name = soup.find("div", class_="c-headerEquipment__contentCol--title")
                if name:
                    name = name.find("h2").text
                else:
                    raise ValueError("Can't find equipment name on page")

                image_divs = soup.findAll("div", class_="c-headerEquipment__slider--slides__img")
                image_links = []
                if image_divs:
                    image_links = [d.find("img").get("src") for d in image_divs]
                else:
                    raise ValueError("Can't find equipment images on page")

                category_divs = soup.find("div", class_="c-headerEquipment__contentCol--equipmentInfos")
                categories = []
                if category_divs:
                    categories = [a.text.replace("\n", "").strip() for a in category_divs.findAll("a")]
                else:
                    raise ValueError("Can't find equipment categories on tge page")

                description = soup.find("div", class_="Editable")
                if description:
                    description = ' '.join([l for l in description.text.split("\n") if l != ""])
                else:
                    raise ValueError("Can't find equipment description on the page")

                scraped_equipment = ScrapedEquipment(
                    name=name,
                    image_links=image_links,
                    mpn=None,
                    skus=None,
                    brands=["Atlantis"],
                    categories=categories,
                    description=description,
                )


In [73]:
scraped_equipment

{'name': 'Biceps curl',
 'image_links': ['https://atlantisstrength.com/app/uploads/2022/01/b157_principal-609x623.png',
  'https://atlantisstrength.com/app/uploads/2022/01/b157_4.png',
  'https://atlantisstrength.com/app/uploads/2022/01/b157_3-609x671.png',
  'https://atlantisstrength.com/app/uploads/2022/01/b157_2.png'],
 'mpn': None,
 'skus': [],
 'brands': ['Atlantis'],
 'categories': ['Arms', 'Selectorized Equipment', 'Precision Series'],
 'description': 'Adjustable sternum pad allows proper axis of rotation alignment. Lever arm is counterbalanced for lower starting resistance. Strategically tapered elbow pads improve durability without compromising comfort. Self-aligning handle bar accommodates a wide variety of gripping positions. Variable resistance cam accurately reflects biceps’ strength curve'}